# Učinki

## Branje vrstic v Jupytru

Ob klicu vgrajene funkcije `read_line` se Jupyter obesi. Zato namesto nje (in `read_int`) definiramo spodnji dve funkciji, ki uporabljata funkcijo `Jupyter_comm.Stdin.read_line`, ki v Jupytru odpre okno z vnosom. Za več informacij glej: <https://github.com/akabe/ocaml-jupyter/issues/162>.

In [15]:
#use "topfind";;
#require "jupyter.comm";;
let read_line () = let str = Jupyter_comm.Stdin.read_line "" in print_endline str; str
let read_int () = read_line () |> int_of_string

- : unit = ()


Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


val read_line : unit -> string = <fun>


val read_int : unit -> int = <fun>


## Vgrajeni učinki v OCamlu

In [16]:
print_endline "Hello, world!"

Hello, world!


- : unit = ()


In [10]:
List.iter print_endline ["Hello,"; "world!"]

Hello,
world!


- : unit = ()


In [11]:
print_endline "Kako ti je ime?";
let ime = read_line () in
print_endline ("Zdravo, " ^ ime)

Kako ti je ime?
Matij
Zdravo, Matij


- : unit = ()


In [12]:
let popolnoma_nakljucno_stevilo = Random.int 1000000 in
print_endline ("Ob prvem klicu bo " ^ string_of_int popolnoma_nakljucno_stevilo ^ " = 809344")

Ob prvem klicu bo 417672 = 809344


- : unit = ()


In [17]:
(* Definiramo izjemo, ki jo bomo sprožili ob vnosu negativnega števila *)
exception NegativnoStevilo

;;

let ugibaj () =
  try
    (* PREDPRIPRAVA *)
    (* Generatorju psevdonaključnih števil psevdonaključno nastavimo seme. *)
    Random.self_init ();
    (* OCamlu povemo, naj ujame klic za prekinitev programa. *)
    Sys.catch_break true;
    print_string "Do koliko znaš šteti? ";
    (* Z vhodne vrstice preberemo niz in ga pretvorimo v število. *)
    let meja = read_int () in
    (* Izračunamo psevdonaključno število v danih mejah *)
    let izmisljeno_stevilo = 1 + Random.int meja in
    print_endline ("Izmislil sem si število med 1 in " ^ string_of_int meja);

    let preberi_stevilo () =
      let poskus = read_int () in
      if poskus < 0 then
        (* Če je vnešeno število negativno, sprožimo izjemo. *)
        raise NegativnoStevilo
      else
        poskus
    in

    (* GLAVNA ZANKA *)
    (* Definiramo glavno zanko programa *)
    let rec ugibaj () =
      print_string "Katero število sem si izmislil? ";
      let poskus = preberi_stevilo () in
      if izmisljeno_stevilo = poskus then
        print_endline "BRAVO!"
      else if izmisljeno_stevilo < poskus then
        begin
          print_endline "Ne, moje število je manjše";
          ugibaj ()
        end
      else
        begin
          print_endline "Ne, moje število je večje";
          ugibaj ()
        end
    in
    (* Poženemo glavno zanko programa *)
    ugibaj ()

  (* LOVLJENJE IZJEM *)
  with
  | Failure msg when msg = "int_of_string" ->
    print_endline "Če ne veš, kako se zapiše števila, tole verjetno nima smisla..."
  | Sys.Break ->
    print_endline "Adijo!"
  | NegativnoStevilo ->
    print_endline "Rekel sem NARAVNO število!"
  | _ ->
    print_endline "Kaj se pa greš?"


exception NegativnoStevilo


val ugibaj : unit -> unit = <fun>


Zgornji program vrstice včasih izpiše z zamikom, tako da ga je bolje poganjati neposredno v terminalu.

In [18]:
ugibaj ()

Do koliko znaš šteti? 
300
Izmislil sem si število med 1 in 300
Katero število sem si izmislil? 
150
Ne, moje število je manjše
Katero število sem si izmislil? 
200
Ne, moje število je manjše
Katero število sem si izmislil? 

Če ne veš, kako se zapiše števila, tole verjetno nima smisla...


- : unit = ()


## Končni avtomati

![Končni avtomat](avtomat.drawio.png)

$$M = (\Sigma, Q, q_0 \in Q, F \subseteq Q, \delta : Q \times \Sigma \to Q)$$

$$M_{primer} = (\{0, 1\}, \{q_0, q_1, q_2\}, q_0, \{q_1\}, \delta)$$

$$
\begin{align*}
\delta(q_0, 0) &= q_1 \\
\delta(q_0, 1) &= q_0 \\
\delta(q_1, 0) &= q_2 \\
\delta(q_1, 1) &= q_1 \\
\delta(q_2, 0) &= q_0 \\
\delta(q_2, 1) &= q_2
\end{align*}
$$

## Končni avtomati s prehodno funkcijo

In [19]:
type stanje = string

type stanje = string


In [20]:
type fsm = {
  stanja : stanje list;
  zacetno_stanje : stanje;
  sprejemna_stanja : stanje list;
  prehodna_funkcija : stanje -> char -> stanje
}

type fsm = {
  stanja : stanje list;
  zacetno_stanje : stanje;
  sprejemna_stanja : stanje list;
  prehodna_funkcija : stanje -> char -> stanje;
}


In [21]:
let ravno_prav_nicel =
  let q0 = "ostanek 0"
  and q1 = "ostanek 1"
  and q2 = "ostanek 2"
  in
  let prehodna_funkcija q c =
    match c with
    | '0' when q = q0 -> q1
    | '0' when q = q1 -> q2
    | '0' when q = q2 -> q0
    | '1' -> q
    | _ -> failwith "Neveljaven prehod"
  in
  {
    stanja = [q0; q1; q2];
    zacetno_stanje = q0;
    sprejemna_stanja = [q1];
    prehodna_funkcija = prehodna_funkcija
  }

val ravno_prav_nicel : fsm =
  {stanja = ["ostanek 0"; "ostanek 1"; "ostanek 2"];
   zacetno_stanje = "ostanek 0"; sprejemna_stanja = ["ostanek 1"];
   prehodna_funkcija = <fun>}


## Končni avtomati s seznamom prehodov

In [20]:
type fsm = {
  stanja : stanje list;
  zacetno_stanje : stanje;
  sprejemna_stanja : stanje list;
  prehodi : (stanje * char * stanje) list
}

type fsm = {
  stanja : stanje list;
  zacetno_stanje : stanje;
  sprejemna_stanja : stanje list;
  prehodi : (stanje * char * stanje) list;
}


In [21]:
let ravno_prav_nicel =
  let q0 = "ostanek 0"
  and q1 = "ostanek 1"
  and q2 = "ostanek 2"
  in
  let prehodi = [
    (q0, '0', q1);
    (q1, '0', q2);
    (q2, '0', q0);
    (q0, '1', q0);
    (q1, '1', q1);
    (q2, '1', q2);
  ]
  in
  {
    stanja = [q0; q1; q2];
    zacetno_stanje = q0;
    sprejemna_stanja = [q1];
    prehodi = prehodi
  }

val ravno_prav_nicel : fsm =
  {stanja = ["ostanek 0"; "ostanek 1"; "ostanek 2"];
   zacetno_stanje = "ostanek 0"; sprejemna_stanja = ["ostanek 1"];
   prehodi =
    [("ostanek 0", '0', "ostanek 1"); ("ostanek 1", '0', "ostanek 2");
     ("ostanek 2", '0', "ostanek 0"); ("ostanek 0", '1', "ostanek 0");
     ("ostanek 1", '1', "ostanek 1"); ("ostanek 2", '1', "ostanek 2")]}


In [22]:
let vsebuje_samo_nicle =
  let ima_enke = "ima 1"
  and nima_enk = "nima 1"
  in
  {
  stanja = [ima_enke; nima_enk];
  zacetno_stanje = nima_enk;
  sprejemna_stanja = [nima_enk];
  prehodi = [
    (nima_enk, '0', nima_enk);
    (nima_enk, '1', ima_enke);
    (ima_enke, '0', ima_enke);
    (ima_enke, '1', ima_enke);
  ]
}

val vsebuje_samo_nicle : fsm =
  {stanja = ["ima 1"; "nima 1"]; zacetno_stanje = "nima 1";
   sprejemna_stanja = ["nima 1"];
   prehodi =
    [("nima 1", '0', "nima 1"); ("nima 1", '1', "ima 1");
     ("ima 1", '0', "ima 1"); ("ima 1", '1', "ima 1")]}


In [23]:
let prehodna_funkcija m q c =
  let (_, _, q') = List.find (fun (q1, c', q2) -> q1 = q && c = c') m.prehodi in
  q'

val prehodna_funkcija : fsm -> stanje -> char -> stanje = <fun>


In [24]:
let ali_sprejema_niz m s =
  let koncno_stanje = String.fold_left (prehodna_funkcija m) m.zacetno_stanje s in
  List.mem koncno_stanje m.sprejemna_stanja

val ali_sprejema_niz : fsm -> string -> bool = <fun>


In [25]:
ali_sprejema_niz ravno_prav_nicel "011110010"

- : bool = true


In [26]:
ali_sprejema_niz vsebuje_samo_nicle "0000000"

- : bool = true


## The Elm Architecture

![The Elm Architecture](tea.drawio.png)